In [5]:
import requests
import json
import time
import xlsxwriter as xw
import os

In [10]:
def crawler_start(page):
    url = ('https://club.jd.com/comment/productPageComments.action?'
           '&productId=100068388533'  # 商品ID
           f'&score=0'  # 0表示所有评论，1表示好评，2表示中评，3表示差评，5表示追加评论
           '&sortType=5'  # 排序类型（通常使用5）
           f'&page={page}'  # 要获取的页面数
           '&pageSize=10'  # 每页评论数
           '&isShadowSku=0'
           '&fold=1')
    # 设置headers以模拟浏览器请求
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0"
    }
    time.sleep(2)
    response = requests.get(url=url,headers=headers)
    return json.loads(response.text)


In [14]:
def parse_data(data):
    items = data['comments']
    for i in items:
        yield (
            i['id'],
            i['creationTime'],
            i['content']
        )

In [6]:
if not os.path.exists("./xlsxdir"):
    os.mkdir("./xlsxdir")

In [8]:
wk = xw.Workbook("./xlsxdir/iphone15.xlsx")
wksheet1 = wk.add_worksheet("sheet1")
wksheet1.activate()
title = ["Id","日期","评论"]
wksheet1.write_row('A1',title)

0

In [15]:
total_page = 4
total_parsed_data_list = []
for i in range(1,total_page+1):
    time.sleep(1.5)
    crawler_data = crawler_start(i)
    parsed_data = parse_data(crawler_data)
    parsed_data_list = list(map(list,parsed_data))
    total_parsed_data_list+=parsed_data_list
    print('第' + str(i) + '页抓取完毕')

In [24]:
for idx,data_list in enumerate(total_parsed_data_list):
    insertData = [str(data_list[0]), data_list[1], data_list[2]]
    row = 'A' + str(idx+1)
    wksheet1.write_row(row,insertData)
wk.close()